In [ ]:
 def prediction(self):
        d = self.get_single_device()
        with tf.device(assign_to_device(d, self.config.controller)):
            x = self.input
            x = tf.reshape(x, [-1, 28, 28, 1])
            x = _relu(_conv('conv1', x, 3, x.get_shape()[-1], 32, 1))
            x = _max_pooling('pool2', _relu(_conv('conv2', x, 3, x.get_shape()[-1], 32, 1)), 2, 2)
            x = tf.contrib.layers.flatten(x)
            self.x = _relu(_fully_connected('fc1', x, 128))
            x1 = lambda: _fully_connected('fc3', x, self.config.n)
            x2 = lambda: _fully_connected('fc4', x, self.config.n)
            x = tf.cond(tf.equal(self.is_task1, tf.constant(True)), x1, x2)
            return x


In [ ]:
def optimize(self):
#         pass
        d = self.get_single_device()
        with tf.device(assign_to_device(d, self.config.controller)):
           
            pred = self.prediction
            cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, 
                labels=tf.one_hot(self.target, self.config.n)))
            if self.freeze:
                optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate,var_list=[self.x] )
            else:
                optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
                  

            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                train_op = optimizer.minimize(cost)
            pdb.set_trace()
            return train_op, cost


In [6]:
a=[2,3]

In [7]:
b=[4,5]

In [8]:
c=[a,b]

In [9]:
print(c)

[[2, 3], [4, 5]]


In [ ]:
 ##Below command will freeze both the weight and biases for the convolution layers
            freeze_conv1  =tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"prediction/conv1")
            freeze_conv2  =tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"prediction/conv2")